# Analyze Kaggle ELO Data

In [1]:
import pandas as panda

import numpy as np

import datetime, time

from matplotlib.pyplot import plot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns


from sklearn.linear_model import LinearRegression, RANSACRegressor, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV,KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

In [2]:
train_data_path = 'all/my_training_data_can_be_used_as_is.csv'
test_data_path = 'all/my_test_data_categorized_can_be_used_as_is.csv'

train_data = panda.read_csv(train_data_path)
test_data = panda.read_csv(test_data_path)
train_data['max_cat_1'] = train_data.max_cat_1.apply(lambda x: 1 if x=='N' else 0)


train_data.head()

,card_id,feature_1,feature_2,feature_3,target,months_passed,total_no_of_transaction,total,total_accepted,total_rejects,mean_installments,max_cat_1,max_cat_2,cat_3
0,C_ID_92A2005557,5,2,1,-0.820283,18,9,-4.135011,8,1,0.014134,1,5,3
1,C_ID_3D0044924F,4,1,0,0.392913,23,9,-4.135011,8,1,1.542135,0,1,3
2,C_ID_D639EDF6CD,2,2,0,0.688056,28,9,-4.135011,8,1,0.000000,1,5,3
3,C_ID_186D6A6901,4,3,0,0.142495,15,9,-4.135011,8,1,1.059524,0,4,3
4,C_ID_CDBD2C0DB2,1,3,0,-0.159749,13,9,-4.135011,8,1,1.284024,0,5,3


In [3]:
outliers = train_data[train_data['target'] < -29]

len(outliers)

2207

In [4]:

## lets check the impact of each column on the target value and select only the ones which have direct impact

# sns.pairplot(train_data[[i for i in train_data.columns.tolist() if i not in['card_id','first_active_month', 'card_id.1','Unnamed:0'] ]])

In [3]:

##based off of the pairplots above we use only the columns -

cols = ['months_passed','total_no_of_transaction',\
#         'total',\
#         'max_cat_2',\
        'total_accepted','total_rejects',\
#         'mean_installments'
       ]

without_outliers = train_data[train_data['target'] > -29]
with_outliers = train_data[train_data['target'] < -29]

round1_x = without_outliers[[i for i in without_outliers.columns.tolist() if i not in ['target','card_id']]]

round1_y = without_outliers[['target']]

round2_x = with_outliers[[i for i in without_outliers.columns.tolist() if i not in ['target','card_id']]]

round2_y = with_outliers[['target']]

# round1_x_train, round1_x_test, round1_y_train, round1_y_test = train_test_split(round1_x, round1_y, test_size = 0.3, random_state = 12)

print(round1_x.shape, round1_y.shape)

(199710, 12) (199710, 1)



## we will fit two models - one with and one without outliers

## both models will be used to predict test targets

## finally , linear regression will be used to predict combined test targets out of the two test targets above

In [4]:

from math import sqrt

def root_mean_square_error(y, y_predicted):
    
    return mean_squared_error(y,y_predicted)
    
scorer = make_scorer(root_mean_square_error, greater_is_better=False)

In [33]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge,BayesianRidge,ElasticNet, Lasso

classifiers = [
    LinearRegression(),
    RANSACRegressor(),
    DecisionTreeRegressor(random_state = 1, criterion = 'mse'),
    RandomForestRegressor(random_state = 1, criterion = 'mse'),
    SGDRegressor(),
#     SVR( kernel = 'rbf'),
#     KernelRidge(),
    Ridge(solver='auto'),
    BayesianRidge(),
    ElasticNet(),
    Lasso()
]


classifier_names = [
            'linear_regression',
            'ransac_regression',
            'decisiontree_regression',
            'randomforest_regression',
            'gradient_descent_regression',
#             'svr',  
#             'kernel_ridge',
            'ridge',
            'bayesian_ridge',
            'elastic_net',
            'lasso',
    
]

classifier_param_grid = [
            
            {},
            {'ransac_regression__min_samples':[50, 75, 125, 200], 'ransac_regression__max_trials':[50, 125, 200], 'ransac_regression__residual_threshold':[5, 10, 14]},
            {'decisiontree_regression__max_depth':[6,7,8,9,10,11]},
            {'randomforest_regression__n_estimators':[1,2,3,5,6]} ,
            {'gradient_descent_regression__max_iter' : [100, 200, 300]},
#             {'svr__C':[1, 5,10]},
#             {'kernel_ridge__alpha':[0.01,0.04,1]},
            {'ridge__alpha':[0.01,0.04,1]},
            {'bayesian_ridge__n_iter':[200,500,600]},
            {'elastic_net__alpha' : [0.01,0.04,1,1.2], 'elastic_net__l1_ratio' :[0.2,0.4,0.5]},
            {'lasso__alpha' : [0.2,0.4,0.6,1],'lasso__max_iter':[200,400,600]}
    
    
]


In [6]:

# classifiers = [
#     LinearRegression(),
#     RANSACRegressor(),
#     DecisionTreeRegressor(random_state = 1, criterion = 'mse'),
#     RandomForestRegressor(random_state = 1, criterion = 'mse'),
#     SGDRegressor(),
#     SVR( kernel = 'rbf'),    
# ]


# classifier_names = [
#             'linear_regression',
#             'ransac_regression',
#             'decisiontree_regression',
#             'randomforest_regression',
#             'gradient_descent_regression',
#             'svr',               
    
# ]

# classifier_param_grid = [
            
#             {},
#             {'ransac_regression__min_samples':[50, 75, 125, 200], 'ransac_regression__max_trials':[50, 125, 200], 'ransac_regression__residual_threshold':[5, 10, 14]},
#             {'decisiontree_regression__max_depth':[6,7,8,9,10,11]},
#             {'randomforest_regression__n_estimators':[1,2,3,5,6]} ,
#             {'gradient_descent_regression__max_iter' : [100, 200, 300]},
#             {'svr__C':[1, 5,10]},
    
# ]


    

class CodeTimer:
    
    """
        Utility custom contextual class for calculating the time 
        taken for a certain code block to execute
    
    """
    def __init__(self, name=None):
        self.name = " '"  + name + "'" if name else ''

    def __enter__(self):
        self.start = time.clock()

    def __exit__(self, exc_type, exc_value, traceback):
        self.took = (time.clock() - self.start) * 1000.0
        time_taken = datetime.timedelta(milliseconds = self.took)
        print('Code block' + self.name + ' took(HH:MM:SS): ' + str(time_taken))



def runGridSearchAndPredict(pipeline, x_train, y_train, x_test, y_test, param_grid, n_jobs = 1, cv = 10, score = 'neg_mean_squared_error'):
    
    response = {}
    training_timer       = CodeTimer('training')
    testing_timer        = CodeTimer('testing')

    with training_timer:

        gridsearch = GridSearchCV(estimator = pipeline, param_grid = param_grid, cv = cv, n_jobs = n_jobs, scoring = score)

        search = gridsearch.fit(x_train,y_train)

        print("Grid Search Best parameters ", search.best_params_)
        print("Grid Search Best score ", search.best_score_)
            
    with testing_timer:
        y_prediction = gridsearch.predict(x_test)
            
    print("Mean squared error score %s" %mean_squared_error(y_test,y_prediction))
    
    response['testing_time'] = testing_timer.took
    response['_y_prediction'] = y_prediction
    response['training_time'] = training_timer.took    
    response['mean_squared_error'] = mean_squared_error(y_test,y_prediction)
    response['root_mean_squared_error'] = search.best_score_
    response['r2_score'] = r2_score(y_test,y_prediction)
    response['best_estimator'] = search.best_estimator_
    
    return response
    


def analyzeRegressionModel(X,y, outliers):

    
    _x_train, _x_test, _y_train, _y_test = train_test_split(X, y, test_size = 0.3, random_state = 2)
    
    
    ## simply ignoring the outliers
#     outlier_x = outliers[[i for i in outliers.columns.tolist() if i not in ['target','card_id']]]
#     outlier_y =  outliers[['target']]
    
#     outlier_x_train, outlier_x_test, outlier_y_train, outlier_y_test = train_test_split(outlier_x, outlier_y, test_size = 0.5, random_state = 2)
    
#     _x_train = panda.concat([_x_train, outlier_x_train])
    
#     _y_train = panda.concat([_y_train, outlier_y_train])
    
#     _x_test = panda.concat([_x_test, outlier_x_test])
    
#     _y_test = panda.concat([_y_test, outlier_y_test])
    
    model_metrics = {}

    for model, model_name, model_param_grid in zip(classifiers, classifier_names, classifier_param_grid):

            pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    (model_name, model)
            ])

            cross_validator = KFold(n_splits = 10, random_state = 12)    
            result = runGridSearchAndPredict(pipeline, _x_train, _y_train, _x_test, _y_test, model_param_grid, cv =cross_validator,score = scorer)
#             result = runGridSearchAndPredict(pipeline, _x_train, _y_train, _x_test, _y_test, model_param_grid,score = scorer)

            _y_prediction = result['_y_prediction']

            model_metrics[model_name] = {}
            model_metrics[model_name]['training_time'] = result['training_time']
            model_metrics[model_name]['testing_time'] = result['testing_time']
            model_metrics[model_name]['r2_score'] = result['r2_score']
            model_metrics[model_name]['mean_squared_error'] = result['mean_squared_error']
            model_metrics[model_name]['root_mean_squared_error'] = result['root_mean_squared_error']
            model_metrics[model_name]['best_estimator'] = result['best_estimator']
            
    return model_metrics
    print('Model metrics are \n :', model_metrics)


In [7]:
round1_x.head()

,feature_1,feature_2,feature_3,months_passed,total_no_of_transaction,total,total_accepted,total_rejects,mean_installments,max_cat_1,max_cat_2,cat_3
0,5,2,1,18,9,-4.135011,8,1,0.014134,1,5,3
1,4,1,0,23,9,-4.135011,8,1,1.542135,0,1,3
2,2,2,0,28,9,-4.135011,8,1,0.000000,1,5,3
3,4,3,0,15,9,-4.135011,8,1,1.059524,0,4,3
4,1,3,0,13,9,-4.135011,8,1,1.284024,0,5,3


In [8]:
round2_x.head()

,feature_1,feature_2,feature_3,months_passed,total_no_of_transaction,total,total_accepted,total_rejects,mean_installments,max_cat_1,max_cat_2,cat_3
64,3,1,1,15,208,-60.798469,191,17,1.033708,0,5,3
92,3,3,1,43,208,-60.798469,191,17,0.000000,1,4,3
125,3,3,1,38,208,-60.798469,191,17,1.467742,0,5,2
151,4,2,0,21,208,-60.798469,191,17,1.000000,0,1,2
225,2,1,0,23,30,-15.394075,28,2,1.016949,0,1,3


In [9]:
model_metrics_without_outliers = analyzeRegressionModel(round1_x, round1_y, with_outliers)

Grid Search Best parameters  {}
Grid Search Best score  -2.9334739969154975
Code block 'training' took(HH:MM:SS): 0:00:03.524536
Code block 'testing' took(HH:MM:SS): 0:00:00.033397
Mean squared error score 2.9712789698909394
Grid Search Best parameters  {'ransac_regression__max_trials': 125, 'ransac_regression__min_samples': 200, 'ransac_regression__residual_threshold': 14}
Grid Search Best score  -2.9333782278977574
Code block 'training' took(HH:MM:SS): 0:03:11.174400
Code block 'testing' took(HH:MM:SS): 0:00:00.023772
Mean squared error score 2.9713594994717663
Grid Search Best parameters  {'decisiontree_regression__max_depth': 6}
Grid Search Best score  -2.9432272430898543
Code block 'training' took(HH:MM:SS): 0:00:57.563157
Code block 'testing' took(HH:MM:SS): 0:00:00.023546
Mean squared error score 2.977002972977157


C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

Grid Search Best parameters  {'randomforest_regression__n_estimators': 6}
Grid Search Best score  -3.6836732091637985
Code block 'training' took(HH:MM:SS): 0:05:36.217389
Code block 'testing' took(HH:MM:SS): 0:00:00.355407
Mean squared error score 3.7644707760562395


C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Grid Search Best parameters  {'gradient_descent_regression__max_iter': 200}
Grid Search Best score  -2.935191185412048
Code block 'training' took(HH:MM:SS): 0:04:45.957035
Code block 'testing' took(HH:MM:SS): 0:00:00.026750
Mean squared error score 2.9722912857065307
Grid Search Best parameters  {'ridge__alpha': 1}
Grid Search Best score  -2.9334327271270704
Code block 'training' took(HH:MM:SS): 0:00:08.404782
Code block 'testing' took(HH:MM:SS): 0:00:00.034786
Mean squared error score 2.9712789612041797


C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Grid Search Best parameters  {'bayesian_ridge__n_iter': 200}
Grid Search Best score  -2.9334299513398054
Code block 'training' took(HH:MM:SS): 0:00:09.978982
Code block 'testing' took(HH:MM:SS): 0:00:00.029367
Mean squared error score 2.9712559410224673
Grid Search Best parameters  {'elastic_net__alpha': 0.01, 'elastic_net__l1_ratio': 0.2}
Grid Search Best score  -2.9334472758472026
Code block 'training' took(HH:MM:SS): 0:00:36.015729
Code block 'testing' took(HH:MM:SS): 0:00:00.028608
Mean squared error score 2.9712233932351357
Grid Search Best parameters  {'lasso__alpha': 0.2, 'lasso__max_iter': 200}
Grid Search Best score  -2.93967826387858
Code block 'training' took(HH:MM:SS): 0:00:34.772058
Code block 'testing' took(HH:MM:SS): 0:00:00.027082
Mean squared error score 2.976979737810643


In [13]:
model_metrics_with_outliers = analyzeRegressionModel(round2_x, round2_y, with_outliers)

Grid Search Best parameters  {}
Grid Search Best score  -1.113070475180238e-28
Code block 'training' took(HH:MM:SS): 0:00:00.107591
Code block 'testing' took(HH:MM:SS): 0:00:00.002331
Mean squared error score 5.048709793414476e-29
Grid Search Best parameters  {'ransac_regression__max_trials': 50, 'ransac_regression__min_samples': 50, 'ransac_regression__residual_threshold': 5}
Grid Search Best score  -1.1117625192752083e-28
Code block 'training' took(HH:MM:SS): 0:00:04.582824
Code block 'testing' took(HH:MM:SS): 0:00:00.001601
Mean squared error score 5.048709793414476e-29
Grid Search Best parameters  {'decisiontree_regression__max_depth': 6}
Grid Search Best score  -1.8175355256292112e-25
Code block 'training' took(HH:MM:SS): 0:00:00.567095
Code block 'testing' took(HH:MM:SS): 0:00:00.001444
Mean squared error score 3.999083027363606e-25


C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

Grid Search Best parameters  {'randomforest_regression__n_estimators': 2}
Grid Search Best score  -1.63578197306629e-26
Code block 'training' took(HH:MM:SS): 0:00:00.793701
Code block 'testing' took(HH:MM:SS): 0:00:00.001104
Mean squared error score 3.155443620884047e-26


C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Grid Search Best parameters  {'gradient_descent_regression__max_iter': 100}
Grid Search Best score  -3.862056007940103e-25
Code block 'training' took(HH:MM:SS): 0:00:02.558974
Code block 'testing' took(HH:MM:SS): 0:00:00.001545
Mean squared error score 3.145905138399536e-25
Grid Search Best parameters  {'ridge__alpha': 0.01}
Grid Search Best score  -1.113070475180238e-28
Code block 'training' took(HH:MM:SS): 0:00:00.299838
Code block 'testing' took(HH:MM:SS): 0:00:00.001191
Mean squared error score 5.048709793414476e-29


In [14]:
best_estimator_without_outliers = model_metrics_without_outliers['linear_regression']['best_estimator']

best_estimator_with_outliers = model_metrics_with_outliers['gradient_descent_regression']['best_estimator']

## at this point we have - outlier_model(M1), without_outlier_model(M2)

## at this point we have - outlier_data(T1), without_outlier_data(T2)

#### 1. we will use M1.fit(T1) and predict on T2. - x1[1]
#### 2. we will use M1.fit(T2) and predict on T1. - x1[2]
#### 3. we will use M2.fit(T1) and predict on T2. - x2[1]
#### 4. we will use M2.fit(T2) and predict on T1. - x2[2]

#### 5. we will use M1.fit(train_data) and predict on test_data - d1_test
#### 6. we will use M2.fit(train_data) and predict on test_data - d2_test

### we will use the above predictions and create a logistic regression model. LR model will train on (x1,x2,train_target) and predict on (d1_test,d2_test)

In [ ]:
best_estimator


## our round_1 estimator is ready.we are simply considering the training data verbatim.

## before we predict on our test data, we would have to read it and then convert the active_first value as well.
## independent variables will be feature_1,_2,_3, months_passed

In [15]:
test_data= panda.read_csv('all/my_test_data_categorized_can_be_used_as_is.csv')

# test_x= test_data[[i for i in test_data.columns.tolist() if i not in ['Unnamed: 0','card_id','card_id.1']]]
test_data.head()

,feature_1,feature_2,feature_3,months_passed,total_no_of_transaction,total,total_accepted,total_rejects,mean_installments,max_cat_1,max_cat_2,cat_3
0,3,3,1,20,9,-4.135011,8,1,2.056338,0,1,2
1,2,3,0,23,9,-4.135011,8,1,1.080460,0,4,2
2,5,1,1,16,9,-4.135011,8,1,3.666667,0,5,3
3,2,1,0,12,9,-4.135011,8,1,1.861111,0,5,3
4,5,1,1,36,9,-4.135011,8,1,1.077586,1,4,3


In [25]:
with_outliers.head(25)

,card_id,feature_1,feature_2,feature_3,target,months_passed,total_no_of_transaction,total,total_accepted,total_rejects,mean_installments,max_cat_1,max_cat_2,cat_3
64,C_ID_8186F3FCC1,3,1,1,-33.219281,15,208,-60.798469,191,17,1.033708,0,5,3
92,C_ID_B9379A30EA,3,3,1,-33.219281,43,208,-60.798469,191,17,0.000000,1,4,3
125,C_ID_E9120F535C,3,3,1,-33.219281,38,208,-60.798469,191,17,1.467742,0,5,2
151,C_ID_65715CB80D,4,2,0,-33.219281,21,208,-60.798469,191,17,1.000000,0,1,2
225,C_ID_AE77D244B6,2,1,0,-33.219281,23,30,-15.394075,28,2,1.016949,0,1,3
248,C_ID_C4262C902E,5,1,1,-33.219281,41,102,-55.732551,91,11,2.360000,0,1,2
254,C_ID_6A2C823E5F,2,1,0,-33.219281,21,102,-55.732551,91,11,1.291667,0,1,2
386,C_ID_98E1CB10B6,2,3,0,-33.219281,18,550,-362.082946,501,49,1.555556,0,3,3
467,C_ID_EB8175721A,4,1,0,-33.219281,21,550,-362.082946,501,49,1.193717,0,1,3
515,C_ID_A2580006BF,4,2,0,-33.219281,32,550,-362.082946,501,49,0.009346,1,5,3


In [26]:
# 1. we will use M1.fit(T1) and predict on T2. - x1[1]
# 2. we will use M1.fit(T2) and predict on T1. - x1[2]
# 3. we will use M1.fit(train_data) and predict on test_data - d1_test

outlier_model = SGDRegressor(max_iter=100)

all_train_x = train_data[[i for i in train_data.columns.tolist() if i not in ['card_id', 'target']]]
all_train_y = train_data[['target']]
train_x1_1 = outlier_model.fit(round1_x,round1_y).predict(round2_x)
outlier_model = SGDRegressor(max_iter=100)
train_x1_2 = outlier_model.fit(round2_x,round2_y).predict(round1_x)
outlier_model = SGDRegressor(max_iter=100)
test_pred_1 = outlier_model.fit(all_train_x, all_train_y).predict(test_data)

print('shape of entire train:', train_data.shape)
print('shape of train_x1_1:', train_x1_1.shape)
print('shape of train_x1_2:', train_x1_2.shape)
print('shape of test_pred_1:', test_pred_1.shape)


print('train_x1_1 \n: ', train_x1_1)
print('train_x1_2 \n: ', train_x1_2)

C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


shape of entire train: (201917, 14)
shape of train_x1_1: (2207,)
shape of train_x1_2: (199710,)
shape of test_pred_1: (123623,)
train_x1_1 
:  [ 1.38154387e+13  1.40681051e+13  1.39820526e+13 ... -9.89699223e+12
  1.17862052e+13  1.19514222e+13]
train_x1_2 
:  [-2.49037389e+12 -1.50106324e+12 -1.71733846e+12 ... -2.35468357e+14
 -2.33571931e+14 -2.35295331e+14]


In [27]:
# 1. we will use M2.fit(T1) and predict on T2. - x2[1]
# 2. we will use M2.fit(T2) and predict on T1. - x2[2]
# 3. we will use M2.fit(train_data) and predict on test_data - d2_test

without_outlier_model = LinearRegression()
train_x2_1 = without_outlier_model.fit(round1_x,round1_y).predict(round2_x)
without_outlier_model = LinearRegression()
train_x2_2 = without_outlier_model.fit(round2_x,round2_y).predict(round1_x)
without_outlier_model = LinearRegression()
test_pred_2 = without_outlier_model.fit(all_train_x, all_train_y).predict(test_data)

print('shape of entire train:', train_data.shape)
print('shape of train_x2_1:', train_x2_1.shape)
print('shape of train_x2_2:', train_x2_2.shape)
print('shape of test_pred_2:', test_pred_2.shape)

print('train_x2_1 \n: ', train_x2_1)
print('train_x2_2 \n: ', train_x2_2)



shape of entire train: (201917, 14)
shape of train_x2_1: (2207, 1)
shape of train_x2_2: (199710, 1)
shape of test_pred_2: (123623, 1)
train_x2_1 
:  [[-0.115646  ]
 [-0.07795413]
 [-0.01296913]
 ...
 [-0.01349364]
 [ 0.07320727]
 [-0.05286602]]
train_x2_2 
:  [[-33.21928095]
 [-33.21928095]
 [-33.21928095]
 ...
 [-33.21928095]
 [-33.21928095]
 [-33.21928095]]


In [28]:


train_x1 = panda.concat([panda.DataFrame(train_x1_1),panda.DataFrame(train_x1_2)], axis=0)
train_x1.columns = ['x1']
print(train_x1.shape, train_x1.head())

train_x2 = panda.concat([panda.DataFrame(train_x2_1),panda.DataFrame(train_x2_2)], axis=0)
train_x2.columns = ['x2']
print(train_x2.shape, train_x2.head())

train_x2.head()

# learnt_test_features = panda.concat([train_x1.x1,train_x2.x2,train_data.target])
# learnt_test_features.columns = ['x1','x2','y']

# print(learnt_test_features.shape, learnt_test_feature.head())

(201917, 1)              x1
0  1.381544e+13
1  1.406811e+13
2  1.398205e+13
3  1.374332e+13
4  5.922526e+11
(201917, 1)          x2
0 -0.115646
1 -0.077954
2 -0.012969
3  0.019534
4 -0.048057


,x2
0,-0.115646
1,-0.077954
2,-0.012969
3,0.019534
4,-0.048057


In [29]:
learnt_new_features_x = panda.concat([train_x1,train_x2],axis=1)
learnt_new_features_y = train_data[['target']]

print(learnt_new_features_x.head(), learnt_new_features_y.head())

             x1        x2
0  1.381544e+13 -0.115646
1  1.406811e+13 -0.077954
2  1.398205e+13 -0.012969
3  1.374332e+13  0.019534
4  5.922526e+11 -0.048057      target
0 -0.820283
1  0.392913
2  0.688056
3  0.142495
4 -0.159749


In [30]:
test_new_features_x1 = panda.DataFrame(test_pred_1)
test_new_features_x1.columns = ['x1']
test_new_features_x2 = panda.DataFrame(test_pred_2)
test_new_features_x2.columns = ['x2']

test_new_features = panda.concat([test_new_features_x1,test_new_features_x2], axis =1)
test_new_features.head(25)

,x1,x2
0,-2.188194e+12,-0.707365
1,-2.390831e+12,-0.752583
2,-1.567596e+12,-0.315852
3,-1.407815e+12,-0.183688
4,-2.939230e+12,-0.497002
5,-1.900047e+12,-0.642251
6,-1.611093e+12,-0.250636
7,-2.133951e+12,-0.185388
8,-1.900048e+12,-0.644023
9,-2.856747e+13,-0.679462


In [31]:
from sklearn.linear_model import LinearRegression
# learnt_new_features_y
lr = LinearRegression()
lr.fit(learnt_new_features_x[['x1','x2']],learnt_new_features_y[['target']])
test_target_ensembled = lr.predict(test_new_features)

test_target_ensembled.shape

(123623, 1)

In [73]:
learnt_new_features_x[['x1','x2']].to_csv('all/learnt_new_features_x.csv')
learnt_new_features_y[['target']].to_csv('all/learnt_new_features_y.csv')
test_new_features.to_csv('all/test_new_features.csv')

In [34]:
ensembled_model = analyzeRegressionModel(learnt_new_features_x[['x1','x2']], learnt_new_features_y[['target']], None)

Grid Search Best parameters  {}
Grid Search Best score  -14.765672148339352
Code block 'training' took(HH:MM:SS): 0:00:00.305601
Code block 'testing' took(HH:MM:SS): 0:00:00.001523
Mean squared error score 14.968838393249007
Grid Search Best parameters  {'ransac_regression__max_trials': 50, 'ransac_regression__min_samples': 50, 'ransac_regression__residual_threshold': 10}
Grid Search Best score  -14.895564434908847
Code block 'training' took(HH:MM:SS): 0:00:48.051118
Code block 'testing' took(HH:MM:SS): 0:00:00.001907
Mean squared error score 15.102640021237924
Grid Search Best parameters  {'decisiontree_regression__max_depth': 6}
Grid Search Best score  -14.803857174244056
Code block 'training' took(HH:MM:SS): 0:00:16.760813
Code block 'testing' took(HH:MM:SS): 0:00:00.003056
Mean squared error score 15.002432739009706


C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

Grid Search Best parameters  {'randomforest_regression__n_estimators': 6}
Grid Search Best score  -22.764274858629
Code block 'training' took(HH:MM:SS): 0:01:41.245847
Code block 'testing' took(HH:MM:SS): 0:00:00.161112
Mean squared error score 23.1298706637656


C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Grid Search Best parameters  {'gradient_descent_regression__max_iter': 300}
Grid Search Best score  -14.766657767655225
Code block 'training' took(HH:MM:SS): 0:01:13.365720
Code block 'testing' took(HH:MM:SS): 0:00:00.002404
Mean squared error score 14.970940933599255
Grid Search Best parameters  {'ridge__alpha': 1}
Grid Search Best score  -14.765672144897072
Code block 'training' took(HH:MM:SS): 0:00:00.673770
Code block 'testing' took(HH:MM:SS): 0:00:00.001562
Mean squared error score 14.968838383966165


C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\somak\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Grid Search Best parameters  {'bayesian_ridge__n_iter': 200}
Grid Search Best score  -14.765768506534158
Code block 'training' took(HH:MM:SS): 0:00:00.845932
Code block 'testing' took(HH:MM:SS): 0:00:00.002232
Mean squared error score 14.96846801398667
Grid Search Best parameters  {'elastic_net__alpha': 0.01, 'elastic_net__l1_ratio': 0.2}
Grid Search Best score  -14.765675612981383
Code block 'training' took(HH:MM:SS): 0:00:03.032800
Code block 'testing' took(HH:MM:SS): 0:00:00.001371
Mean squared error score 14.968683001269069
Grid Search Best parameters  {'lasso__alpha': 0.2, 'lasso__max_iter': 200}
Grid Search Best score  -14.765908394707923
Code block 'training' took(HH:MM:SS): 0:00:03.035593
Code block 'testing' took(HH:MM:SS): 0:00:00.001869
Mean squared error score 14.96806338684774


In [35]:
test_target_ensembled = ensembled_model['lasso']['best_estimator'].predict(test_new_features)

In [36]:
test_target_ensembled

array([-0.39199346, -0.39199346, -0.39199346, ..., -0.39199346,
       -0.39199346, -0.39199346])

In [37]:
## write it to a panda table

temp = panda.read_csv('all/test.csv')
sample_submission_round_1 = temp[['card_id']]

# sample_submission_round_1 = test_data[['card_id']]
sample_submission_round_1['target'] = test_target_ensembled
# sample_submission_round_1['target_without_outliers'] = test_target_without_outliers

print(len(sample_submission_round_1), sample_submission_round_1.head())

sample_submission_round_1.to_csv('round_ensembled_2_submission.csv')

C:\Users\somak\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


123623            card_id    target
0  C_ID_0ab67a22ab -0.391993
1  C_ID_130fd0cbdd -0.391993
2  C_ID_b709037bc5 -0.391993
3  C_ID_d27d835a9f -0.391993
4  C_ID_2b5e3df5c2 -0.391993
